# Importing libraries and reading in the data

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
pd.set_option('display.max_columns', None)

In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
target = pd.read_csv('target.csv')

In [3]:
# display(categorical.head(3))
# display(numerical.head(3))
# display(target.head(3))

In [4]:
data = pd.concat([categorical, numerical, target], axis = 1)
data

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0,0.0
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,0,0.0
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10

# Checking for imbalance in the Target

In [5]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

# X,y and Train, Test Split

In [6]:
X = data.drop(['TARGET_B', 'TARGET_D'], axis = 1)
y = data['TARGET_B']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
y_train

16390    0
53420    0
88542    0
12199    1
33257    0
        ..
21440    0
73349    0
50057    0
5192     0
77708    0
Name: TARGET_B, Length: 76329, dtype: int64

# Treating the imbalance

##### in this lab by upsampling

In [9]:
up = pd.concat([X_train, y_train], axis = 1)

In [10]:
# dropping Target D
data_noD = up.copy()
data_noD

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B
16390,other,34,U,M,1,L,E,B,T,1,87,1,56,4,90,11,89,11,95,11,87,8,1,42.000000,5,8,1,2,18,49,7,3,4,13,8,4162,1091,1915,99,0,1,52,48,88,8,0,3,2,1,0,1,0,0,0,0,0,1,0,1,28,31,33,29,36,38,22,55,32,13,13,49,20,9,5,2,3,27,21,28,16,8,3,1,2,37,34,30,15,4,1,58,13,4,26,140,213,32,23,3,67,65,60,2,2,0,820,816,4,4,32,68,86,14,2,29,57,48,23,75,25,6,1,5,28,17,7,17,66,4,2,6,1,1,1,11,65,98,0,4,23,44,46,12,4,30,38,0,5,8,55,0,24,64,92,97,3440.0,185.0,691.0,386,438,405,462,18655,9,12,21,31,21,5,1,0,0,6,7,17,30,32,6,2,0,0,4,2,53,4,83,17,40,89,8,0,0,0,0,0,1,1,0,6,16,18,1,61,82,94,69,93,65,50,34,82,99,3,37,20,9,7,11,0,1,2,1,8,3,0,2,1,0,8,35,3,2,4,8,3,6,1,0,6,3,12,9,3,4,13,2,75,3,0,155,1,6,14,23,8,35,13,3,23,2,13,11,1,2,0,10,18,1,49,8,7,30,1,4,2,11,0,0,5,1,0,1,0,0,2,1,0,5,40,91,2,2,5,99,59,10,1,5,2,57,72,91,99,0,4,3,13,0,86,1,0,0,99,0,99,99,98,7,2,2,3,25,65,4,11,110.0,15,6,3.0,12.0,11.0,21,7.333333,48062,0,1,17,0
53420,NC,49,H,M,3,L,E,B,R,2,94,1,0,2,93,10,94,12,96,3,93,10,0,61.611649,5,9,0,0,23,29,37,5,5,1,9,7617,2112,2688,0,8,92,48,52,72,27,0,0,1,0,0,0,0,0,0,0,0,0,0,0,32,42,46,35,45,49,27,34,43,23,14,20,20,15,12,10,9,14,13,33,22,17,26,10,24,19,33,48,30,12,5,60,8,10,23,194,278,78,78,2,6,4,2,1,1,0,518,565,3,2,78,22,95,5,3,41,79,64,32,89,11,9,2,8,9,18,2,14,43,15,8,16,3,1,2,7,21,52,0,1,46,52,55,15,5,5,0,15,13,3,2,3,1,5,24,67,1520.0,279.0,517.0,261,307,313,349,11269,27,20,16,20,12,2,1,1,1,19,21,18,24,14,3,1,1,1,30,6,26,11,34,66,6,76,17,0,0,0,0,0,4,2,1,21,20,25,8,64,66,72,60,70,58,78,58,75,83,6,9,8,3,9,12,0,1,9,5,17,16,5,6,7,0,10,33,2,2,4,13,3,3,2,1,5,10,3,2,5,6,1,9,76,4,0,120,15,22,34,13,7,7,3,5,24,1,21,8,0,0,0,11,23,1,27,18,38,9,0,9,0,7,0,0,5,1,0,0,0,0,1,0,0,0,84,98,1,0,1,91,65,28,12,25,2,9,20,42,61,39,0,0,1,16,35,30,0,17,45,55,32,97,90,8,3,6,6,12,30,6,13,47.0,6,0,2.0,10.0,10.0,6,7.833333,18920,1,3,44,0
88542,IL,24,U,M,1,L,E,A,C,1,89,1,37,7,90,2,96,3,96,3,89,12,1,60.000000,2,9,12,0,21,58,0,7,0,4,9,1748,442,467,99,0,0,50,50,79,15,0,5,5,2,0,0,1,0,1,0,3,1,0,1,29,39,41,29,40,43,36,29,45,26,14,19,37,19,8,3,2,10,12,37,25,16,8,1,4,4,19,78,56,27,10,6

In [11]:
# categories with and without B = 1
category_0 = data_noD[data_noD['TARGET_B'] == 0]
category_1 = data_noD[data_noD['TARGET_B'] == 1]

In [12]:
# upsampling cat. 1 to match length of cat. 0
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [13]:
print(category_0.shape)
print(category_1_oversampled.shape)

(72464, 338)
(72464, 338)


In [14]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [15]:
data_upsampled

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B
16390,other,34,U,M,1,L,E,B,T,1,87,1,56,4,90,11,89,11,95,11,87,8,1,42.000000,5,8,1,2,18,49,7,3,4,13,8,4162,1091,1915,99,0,1,52,48,88,8,0,3,2,1,0,1,0,0,0,0,0,1,0,1,28,31,33,29,36,38,22,55,32,13,13,49,20,9,5,2,3,27,21,28,16,8,3,1,2,37,34,30,15,4,1,58,13,4,26,140,213,32,23,3,67,65,60,2,2,0,820,816,4,4,32,68,86,14,2,29,57,48,23,75,25,6,1,5,28,17,7,17,66,4,2,6,1,1,1,11,65,98,0,4,23,44,46,12,4,30,38,0,5,8,55,0,24,64,92,97,3440.0,185.0,691.0,386,438,405,462,18655,9,12,21,31,21,5,1,0,0,6,7,17,30,32,6,2,0,0,4,2,53,4,83,17,40,89,8,0,0,0,0,0,1,1,0,6,16,18,1,61,82,94,69,93,65,50,34,82,99,3,37,20,9,7,11,0,1,2,1,8,3,0,2,1,0,8,35,3,2,4,8,3,6,1,0,6,3,12,9,3,4,13,2,75,3,0,155,1,6,14,23,8,35,13,3,23,2,13,11,1,2,0,10,18,1,49,8,7,30,1,4,2,11,0,0,5,1,0,1,0,0,2,1,0,5,40,91,2,2,5,99,59,10,1,5,2,57,72,91,99,0,4,3,13,0,86,1,0,0,99,0,99,99,98,7,2,2,3,25,65,4,11,110.0,15,6,3.0,12.0,11.0,21,7.333333,48062,0,1,17,0
53420,NC,49,H,M,3,L,E,B,R,2,94,1,0,2,93,10,94,12,96,3,93,10,0,61.611649,5,9,0,0,23,29,37,5,5,1,9,7617,2112,2688,0,8,92,48,52,72,27,0,0,1,0,0,0,0,0,0,0,0,0,0,0,32,42,46,35,45,49,27,34,43,23,14,20,20,15,12,10,9,14,13,33,22,17,26,10,24,19,33,48,30,12,5,60,8,10,23,194,278,78,78,2,6,4,2,1,1,0,518,565,3,2,78,22,95,5,3,41,79,64,32,89,11,9,2,8,9,18,2,14,43,15,8,16,3,1,2,7,21,52,0,1,46,52,55,15,5,5,0,15,13,3,2,3,1,5,24,67,1520.0,279.0,517.0,261,307,313,349,11269,27,20,16,20,12,2,1,1,1,19,21,18,24,14,3,1,1,1,30,6,26,11,34,66,6,76,17,0,0,0,0,0,4,2,1,21,20,25,8,64,66,72,60,70,58,78,58,75,83,6,9,8,3,9,12,0,1,9,5,17,16,5,6,7,0,10,33,2,2,4,13,3,3,2,1,5,10,3,2,5,6,1,9,76,4,0,120,15,22,34,13,7,7,3,5,24,1,21,8,0,0,0,11,23,1,27,18,38,9,0,9,0,7,0,0,5,1,0,0,0,0,1,0,0,0,84,98,1,0,1,91,65,28,12,25,2,9,20,42,61,39,0,0,1,16,35,30,0,17,45,55,32,97,90,8,3,6,6,12,30,6,13,47.0,6,0,2.0,10.0,10.0,6,7.833333,18920,1,3,44,0
88542,IL,24,U,M,1,L,E,A,C,1,89,1,37,7,90,2,96,3,96,3,89,12,1,60.000000,2,9,12,0,21,58,0,7,0,4,9,1748,442,467,99,0,0,50,50,79,15,0,5,5,2,0,0,1,0,1,0,3,1,0,1,29,39,41,29,40,43,36,29,45,26,14,19,37,19,8,3,2,10,12,37,25,16,8,1,4,4,19,78,56,27,10,6

# Random Forest before Feature Selection

##### with encoding the categoricals because otherwise it wont work for some reason...

In [16]:
X = data_upsampled.drop(['TARGET_B'], axis = 1)
y = data_upsampled['TARGET_B']

In [17]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
X_train_num = X.select_dtypes(np.number).reset_index()
X_test_num = X_test.select_dtypes(np.number).reset_index()
X_train_cat = X.select_dtypes(object)
X_test_cat = X_test.select_dtypes(object)

In [19]:
encoder = OneHotEncoder(drop='first').fit(X_train_cat) 
cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
X_train_cat_encode = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
X_test_cat_encode = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)

In [20]:
# display(X_train_num)
# display(X_test_num)
# display(X_train_cat_encode)
# display(X_test_cat_encode)

In [55]:
X_train_new = pd.concat([X_train_num, X_train_cat_encode], axis=1)
X_test_new = pd.concat([X_test_num, X_test_cat_encode], axis=1)
X_whole = pd.concat([X_train_new, X_test_new], axis=0)

In [22]:
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train_new, y)
print(clf.score(X_train_new, y))
print(clf.score(X_test_new, y_test))

y_pred = clf.predict(X_test_new)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6218950099359682
0.6090761410679663


0    18105
1      978
Name: TARGET_B, dtype: int64

array([[11068,  7037],
       [  423,   555]], dtype=int64)

# Now feature selection

##### Variance Threshold

In [23]:
numerical

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

In [24]:
numericals = data_upsampled.select_dtypes(np.number)
# numericals = numerical.drop(['TARGET_B'], axis = 1)
# first: scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numericals)

# setting threshold
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.07
sel = VarianceThreshold(threshold=(var_threshold))

# running the selector
sel = sel.fit(numerical_scaled) # train selector with scaled numericals
temp = sel.transform(numerical_scaled) # transform scaled numericals with selector
temp = pd.DataFrame(temp) # storing the columns, that will be kept in a temporary dataframe
print(numerical_scaled.shape) # comparing scaled data with
print(temp.shape)             # columns we will keep

# for removing the columns that are not in temp dataframe
sel.variances_ > var_threshold 
sel.get_support() 
var_list = list(sel.get_support())
var_list # list contains true and false values wether column was above or below threshold
list(zip(numerical.columns, var_list)) # put booleans and column names together (automatically right order)
dropping = [col[0] for col in zip(numerical.columns, var_list) if col[1] == False]
dropping # list with columns to drop

(144928, 331)
(144928, 39)


['INCOME',
 'WEALTH1',
 'HIT',
 'MALEMILI',
 'VIETVETS',
 'LOCALGOV',
 'POP901',
 'POP903',
 'POP90C1',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HU3',
 'HU5',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2',
 'HVP3',
 'HVP4',
 'HVP5',
 'HVP6',
 'HUR1',
 'HUR2',
 'RH

In [25]:
data_upsampled2 = data_upsampled.drop(dropping, axis = 1).copy()
data_upsampled2

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,MALEVET,WWIIVETS,STATEGOV,FEDGOV,WEALTH2,POP902,POP90C2,POP90C3,ETH7,ETH11,ETH12,ETH13,HU4,HUPA4,HUPA5,HUPA6,HUPA7,RP1,IC9,IC10,IC11,IC13,OCC13,EIC1,HC21,MHUC1,MHUC2,CARDPROM,CARDPM12,CARDGIFT,MINRAMNT,MAXRAMNT,TARGET_B
16390,other,34,U,M,1,L,E,B,T,1,87,1,56,4,90,11,89,11,95,11,87,8,1,42.000000,18,7,4,13,8,1091,0,1,0,0,0,0,14,5,8,55,0,24,31,21,5,0,2,1,98,7,2,25,4,6,3.0,12.0,0
53420,NC,49,H,M,3,L,E,B,R,2,94,1,0,2,93,10,94,12,96,3,93,10,0,61.611649,23,37,5,1,9,2112,8,92,0,0,0,0,5,13,3,2,3,1,20,12,2,1,6,7,90,8,3,12,6,0,2.0,10.0,0
88542,IL,24,U,M,1,L,E,A,C,1,89,1,37,7,90,2,96,3,96,3,89,12,1,60.000000,21,0,0,4,9,442,0,0,0,1,0,3,1,3,0,0,0,93,20,43,15,3,6,0,99,10,4,22,5,10,3.0,12.0,0
33257,IN,49,U,M,3,L,G,C,R,2,96,1,0,2,96,2,96,2,96,2,96,2,1,61.611649,27,19,2,3,9,915,1,99,0,0,0,0,5,16,1,2,3,0,19,22,2,0,3,10,97,9,2,6,5,1,25.0,25.0,0
51036,other,45,U,M,2,L,D,D,R,2,86,1,51,10,91,3,95,4,95,9,89,6,0,46.000000,34,38,5,3,1,656,0,99,0,0,0,0,11,22,1,6,0,0,12,5,1,0,3,19,96,9,2,22,4,6,2.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25725,WI,39,U,F,3,L,E,B,T,2,95,1,57,1,95,7,96,1,96,1,95,7,0,41.000000,27,24,0,1,9,403,79,21,0,0,0,0,1,12,12,32,0,12,25,14,3,0,6,3,98,8,2,8,6,1,5.0,11.0,1
41223,IN,14,H,F,3,L,F,B,S,1,96,1,40,1,95,10,95,10,95,10,95,10,2,58.000000,27,16,3,1,9,1325,0,6,0,0,0,1,4,3,1,0,0,87,19,37,18,2,2,0,99,9,2,5,3,0,15.0,15.0,1
79637,WA,28,U,M,3,L,E,D,C,2,86,1,25,1,87,6,93,6,95,12,87,2,2,73.000000,43,46,4,10,2,734,0,99,0,0,0,1,51,15,1,0,3,9,18,15,1,0,4,6,99,9,2,21,4,8,5.0,25.0,1
56326,IL,11,U,F,2,L,D,A,S,1,91,1,34,7,95,5,94,11,95,12,91,1,2,63.000000,24,11,1,1,8,534,0,0,0,1,0,1,2,7,0,7,0,54,30,35,8,1,2,3,99,10,3,23,6,8,4.0,11.0,1


In [26]:
X_train_up_vt = X_train_new.drop(dropping, axis = 1).copy()
X_test_vt = X_test_new.drop(dropping, axis = 1).copy()
y
y_test

51495    0
28805    0
81674    0
1833     0
8616     1
        ..
16697    0
45395    0
46816    1
31664    0
50736    0
Name: TARGET_B, Length: 19083, dtype: int64

In [27]:
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train_up_vt, y)
print(clf.score(X_train_up_vt, y))
print(clf.score(X_test_vt, y_test))

y_pred = clf.predict(X_test_vt)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6069082578935747
0.6048315254414924


0    18105
1      978
Name: TARGET_B, dtype: int64

array([[10994,  7111],
       [  430,   548]], dtype=int64)

In [28]:
# tried a few different thresholds, all give abaout the same results for randomforest,
# which in turn are very similar (difference in the thousands(.000)) to before

##### PCA

In [29]:
# numerical_scaled
# X_train_cat_encode
# X_test_cat_encode

In [30]:
X_train_up_pca = X_train_new.copy()
X_test_pca = X_test_new.copy()
y
y_test

51495    0
28805    0
81674    0
1833     0
8616     1
        ..
16697    0
45395    0
46816    1
31664    0
50736    0
Name: TARGET_B, Length: 19083, dtype: int64

In [31]:
pca = PCA(0.85)
pca.fit(X_train_new)

PCA(n_components=0.85)

In [32]:
X_train_up_pca = pca.transform(X_train_up_pca)
X_test_pca = pca.transform(X_test_pca)

In [33]:
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train_up_pca, y)
print(clf.score(X_train_up_pca, y))
print(clf.score(X_test_pca, y_test))

y_pred = clf.predict(X_test_pca)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.5472855486862442
0.6718545302101346


0    18105
1      978
Name: TARGET_B, dtype: int64

array([[12464,  5641],
       [  621,   357]], dtype=int64)

In [34]:
clf.predict_proba(X_test_pca)

array([[0.52554319, 0.47445681],
       [0.45773571, 0.54226429],
       [0.52839562, 0.47160438],
       ...,
       [0.52599312, 0.47400688],
       [0.52491406, 0.47508594],
       [0.45092875, 0.54907125]])

In [58]:
# still improved on test data...
X_test_pca

array([[  6383.792677  ,   3793.93148793],
       [-57743.34117644, -18999.0714069 ],
       [-18911.84782898,  33931.20237817],
       ...,
       [ 10920.23934854,   -880.49562605],
       [ 17247.50852885, -16021.20137369],
       [-67482.88187163,   2925.23274151]])

In [57]:
pred = clf.predict(X_whole)

C:\Users\Customer\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 62 features, but RandomForestClassifier is expecting 2 features as input.

In [53]:
Target_B_pred = pd.DataFrame(y_pred)
Target_B_pred

,0
0,0
1,1
2,0
3,0
4,0
...,...
19078,0
19079,0
19080,0
19081,0


## Lab | Final regression model in "Health Care for All" Case

In [36]:
from sklearn import linear_model
data_pred_d = data[data['TARGET_B'] == 1]
data_pred_d

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
20,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,2,62.000000,3,8,10,2,25,40,27,11,4,1,9,2707,672,929,99,0,0,45,55,97,1,0,2,1,0,0,0,0,1,0,0,1,0,0,0,35,42,43,37,47,50,27,34,43,23,9,20,27,16,8,6,15,15,13,37,21,14,16,8,15,23,30,47,29,11,2,55,10,13,22,190,267,89,49,1,10,10,8,8,8,0,902,960,8,8,88,12,97,3,11,42,72,60,34,79,21,8,1,7,12,21,5,21,58,17,0,1,0,0,4,35,74,99,0,1,64,63,62,14,4,2,8,0,5,1,7,0,95,96,96,96,5120.0,107.0,613.0,468,501,540,580,20328,6,8,19,23,23,11,7,1,2,4,6,13,27,25,14,8,1,1,14,2,66,1,61,39,20,74,17,2,2,0,0,0,2,4,1,75,20,21,1,67,71,82,62,80,60,80,65,99,0,1,21,20,8,11,17,1,1,10,0,5,3,2,1,0,0,3,23,5,2,4,12,7,3,2,1,14,12,7,5,11,4,1,4,70,10,0,140,5,3,24,21,9,32,7,3,23,3,16,7,1,2,0,12,25,3,40,20,27,7,0,0,2,12,0,0,3,1,4,1,0,0,0,4,0,1,58,95,1,0,4,95,69,23,4,13,2,17,35,91,99,1,12,3,93,0,5,1,0,1,99,1,97,99,99,9,2,6,3,26,65,5,12,61.00,15,10,2.00,7.0,5.0,12,4.066667,82943,1,3,3,1,4.0
30,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,0,61.611649,5,9,0,1,37,58,16,8,1,5,9,2147,591,640,99,0,0,49,51,94,2,0,3,5,1,0,1,1,0,0,0,3,0,0,2,29,36,37,27,38,40,37,46,37,17,8,34,34,15,6,2,1,18,21,31,19,11,5,0,3,7,23,71,47,15,6,77,5,1,17,286,334,99,99,0,0,0,0,0,0,0,1002,1166,7,5,97,3,96,4,3,63,92,88,59,97,3,4,1,3,5,4,1,30,60,4,1,2,0,4,15,50,94,99,2,0,91,70,72,20,4,0,0,0,3,0,0,0,78,83,99,99,2920.0,201.0,618.0,633,638,652,663,19703,2,5,1,17,48,19,7,0,1,3,4,0,16,51,19,7,0,1,5,0,65,3,74,26,25,89,9,0,0,0,0,0,1,2,1,82,34,42,22,70,82,98,67,97,66,68,61,99,99,0,34,23,4,10,12,0,2,9,0,4,1,1,0,0,4,2,25,5,2,4,8,8,8,0,3,10,8,9,4,8,1,5,5,76,4,1,160,0,1,12,25,10,39,13,5,33,6,25,7,1,1,0,19,37,2,58,11,16,10,0,4,1,15,0,0,3,2,0,2,0,0,0,1,0,3,48,97,1,1,1,99,91,24,3,6,6,57,85,95,95,5,0,0,75,0,20,0,0,5,96,4,90,99,99,12,4,1,6,10,24,0,2,68.00,11,6,2.00,10.0,7.0,9,6.181818,190313,1,3,14,1,7.0
45,other,24,H,F,3,L,D,C,C,1,86,1,31,10,93,12,94,4,96,2,87,4,0,66.000000,5,9,5,0,33,24,39,6,5,1,9,2160,683,900,89,0,11,48,52,99,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,42,47,49,40,48,50,20,33,42,25,7,14,23,21,17,13,5,14,13,33,22,18,24,7,22,22,43,35,19,6,1

## LM with all columns

In [37]:
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error


y_d = data_pred_d['TARGET_D']
X_d = data_pred_d.drop(['TARGET_D', 'TARGET_B'], axis=1)


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0.2)


X_train_num_d = X_train_d.select_dtypes(np.number).reset_index(drop=True)
X_test_num_d = X_test_d.select_dtypes(np.number).reset_index(drop=True)
X_train_cat_d = X_train_d.select_dtypes(object)
X_test_cat_d = X_test_d.select_dtypes(object)


MinMaxtransformer = MinMaxScaler().fit(X_train_num_d)
X_train_num_normalized = pd.DataFrame(MinMaxtransformer.transform(X_train_num_d),columns=X_train_num_d.columns)
X_test_num_normalized = pd.DataFrame(MinMaxtransformer.transform(X_test_num_d),columns=X_train_num_d.columns)

encoder = OneHotEncoder(drop='first').fit(X_train_cat_d) 
cols = encoder.get_feature_names_out(input_features=X_train_cat_d.columns)
X_train_cat_encode_d = pd.DataFrame(encoder.transform(X_train_cat_d).toarray(),columns=cols)
X_test_cat_encode_d = pd.DataFrame(encoder.transform(X_test_cat_d).toarray(),columns=cols)

X_train_new_d = pd.concat([X_train_num_normalized, X_train_cat_encode_d], axis=1)
X_test_new_d = pd.concat([X_test_num_normalized, X_test_cat_encode_d], axis=1)

In [38]:
lm = linear_model.LinearRegression()
lm.fit(X_train_new_d,y_train_d)

predictions_d = lm.predict(X_train_new_d)
print(r2_score(y_train_d, predictions_d))
predictions_test_d = lm.predict(X_test_new_d)
print(r2_score(y_test_d, predictions_test_d))
mse = mean_squared_error(y_test_d,predictions_test_d)
rmse = np.sqrt(mean_squared_error(y_test_d,predictions_test_d))
print(rmse)
print(mse)

0.6024058069914243
-0.20281725412945106
11.383536397789376
129.58490091979553


## LM features selected by variance

In [39]:
data_pred_d_vt = data[data['TARGET_B'] == 1]
data_pred_d_vt = data_pred_d_vt.drop(dropping, axis = 1).copy()
data_pred_d_vt = data_pred_d_vt.drop(['STATE'], axis = 1)

In [40]:
data_pred_d_vt

,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,MALEVET,WWIIVETS,STATEGOV,FEDGOV,WEALTH2,POP902,POP90C2,POP90C3,ETH7,ETH11,ETH12,ETH13,HU4,HUPA4,HUPA5,HUPA6,HUPA7,RP1,IC9,IC10,IC11,IC13,OCC13,EIC1,HC21,MHUC1,MHUC2,CARDPROM,CARDPM12,CARDGIFT,MINRAMNT,MAXRAMNT,TARGET_B,TARGET_D
20,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,2,62.000000,25,27,4,1,9,672,0,0,0,0,0,1,3,5,1,7,0,95,23,23,11,1,1,0,99,9,2,26,5,10,2.00,7.0,1,4.0
30,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,0,61.611649,37,16,1,5,9,591,0,0,0,0,0,3,4,3,0,0,0,78,17,48,19,0,0,0,99,12,4,10,0,6,2.00,10.0,1,7.0
45,24,H,F,3,L,D,C,C,1,86,1,31,10,93,12,94,4,96,2,87,4,0,66.000000,33,39,5,1,9,683,0,11,0,0,0,0,3,2,2,0,0,39,10,24,17,3,1,1,97,11,3,31,5,14,3.00,6.0,1,5.0
78,13,H,F,2,L,F,A,S,1,90,1,28,7,90,1,95,3,95,11,90,1,0,69.000000,34,54,3,1,9,3736,0,0,1,0,0,2,6,20,10,6,0,92,11,15,13,6,1,1,99,8,3,22,6,5,5.00,17.0,1,13.0
93,18,H,M,3,L,E,A,S,2,92,1,24,10,92,9,95,9,95,9,92,9,1,73.000000,21,8,4,11,7,418,0,0,0,0,0,0,2,10,0,0,0,69,35,20,10,0,6,0,99,7,2,19,6,8,5.00,12.0,1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95298,36,H,F,3,L,F,A,T,2,86,1,53,4,89,6,96,1,96,1,86,8,2,45.000000,45,37,2,3,2,671,99,1,0,0,0,6,12,23,11,8,0,13,17,7,1,1,7,0,95,6,2,33,6,16,0.07,17.0,1,20.0
95309,12,H,F,3,L,F,B,S,1,94,1,47,1,93,10,94,2,95,12,93,10,0,51.000000,32,24,5,6,6,2324,0,0,1,0,0,6,13,12,3,13,0,58,28,27,6,0,2,1,98,12,2,13,4,2,5.00,15.0,1,15.0
95398,11,H,F,3,L,G,B,S,1,86,1,11,10,89,6,95,11,96,2,87,11,0,86.000000,32,26,1,0,9,651,0,0,0,0,0,1,2,2,1,22,0,30,21,31,12,1,3,1,99,8,4,29,6,4,5.00,25.0,1,3.0
95403,49,H,F,2,L,F,D,R,2,90,1,40,1,90,3,93,12,96,1,90,3,0,58.000000,24,20,1,2,5,450,1,99,0,0,0,0,3,9,3,7,2,0,22,12,4,0,3,16,97,7,4,22,4,6,3.00,20.0,1,10.0


In [41]:
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error


y_d = data_pred_d_vt['TARGET_D']
X_d = data_pred_d_vt.drop(['TARGET_D', 'TARGET_B'], axis=1)


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0.2)


X_train_num_d = X_train_d.select_dtypes(np.number).reset_index(drop=True)
X_test_num_d = X_test_d.select_dtypes(np.number).reset_index(drop=True)
X_train_cat_d = X_train_d.select_dtypes(object)
X_test_cat_d = X_test_d.select_dtypes(object)


MinMaxtransformer = MinMaxScaler().fit(X_train_num_d)
X_train_num_normalized = pd.DataFrame(MinMaxtransformer.transform(X_train_num_d),columns=X_train_num_d.columns)
X_test_num_normalized = pd.DataFrame(MinMaxtransformer.transform(X_test_num_d),columns=X_train_num_d.columns)

encoder = OneHotEncoder(drop='first').fit(X_train_cat_d) 
cols = encoder.get_feature_names_out(input_features=X_train_cat_d.columns)
X_train_cat_encode_d = pd.DataFrame(encoder.transform(X_train_cat_d).toarray(),columns=cols)
X_test_cat_encode_d = pd.DataFrame(encoder.transform(X_test_cat_d).toarray(),columns=cols)

X_train_new_d = pd.concat([X_train_num_normalized, X_train_cat_encode_d], axis=1)
X_test_new_d = pd.concat([X_test_num_normalized, X_test_cat_encode_d], axis=1)

In [42]:
X_test_new_d

,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,MALEVET,WWIIVETS,STATEGOV,FEDGOV,WEALTH2,POP902,POP90C2,POP90C3,ETH7,ETH11,ETH12,ETH13,HU4,HUPA4,HUPA5,HUPA6,HUPA7,RP1,IC9,IC10,IC11,IC13,OCC13,EIC1,HC21,MHUC1,MHUC2,CARDPROM,CARDPM12,CARDGIFT,MINRAMNT,MAXRAMNT,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.807692,0.5,0.333333,0.454545,0.0,0.655914,0.181818,0.454545,0.181818,0.857143,0.000000,0.0,0.454545,0.761905,0.181818,0.000000,0.351064,0.481013,0.434343,0.020833,0.016667,0.222222,0.019229,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.326531,0.090909,0.014085,0.000000,0.108696,0.050505,0.236364,0.101010,0.081081,0.000000,0.20,0.028571,0.898990,0.238095,0.4,0.326531,0.235294,0.097561,0.011111,0.010050,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.230769,1.0,0.000000,0.090909,0.0,0.494624,0.363636,0.090909,0.454545,0.571429,0.090909,0.5,0.000000,0.571429,0.000000,0.000026,0.510638,0.392405,0.686869,0.041667,0.000000,1.000000,0.024752,0.000000,0.000000,0.018182,0.0,0.0,0.010870,0.040816,0.060606,0.000000,0.070707,0.000000,0.737374,0.236364,0.111111,0.270270,0.285714,0.00,0.014286,1.000000,0.714286,0.8,0.306122,0.176471,0.097561,0.006667,0.007035,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.326923,1.0,0.333333,0.545455,0.0,0.559140,0.000000,0.545455,0.818182,0.857143,0.727273,0.0,0.727273,0.809524,0.818182,0.000000,0.446809,0.303797,0.080808,0.041667,0.066667,0.777778,0.024885,0.000000,0.000000,0.000000,0.0,0.0,0.010870,0.020408,0.050505,0.028169,0.000000,0.000000,0.909091,0.400000,0.393939,0.216216,0.095238,0.00,0.028571,1.000000,0.476190,0.8,0.387755,0.352941,0.097561,0.022222,0.014070,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.750000,1.0,0.333333,0.000000,0.0,0.430108,0.909091,0.818182,1.000000,0.571429,0.090909,0.5,0.000000,0.523810,0.636364,0.000051,0.563830,0.481013,0.353535,0.000000,0.016667,0.555556,0.015104,0.565657,0.444444,0.000000,0.0,0.0,0.000000,0.081633,0.060606,0.056338,0.010101,0.000000,0.060606,0.400000,0.141414,0.378378,0.047619,0.32,0.014286,1.000000,0.333333,0.6,0.632653,0.352941,0.560976,0.006667,0.003015,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.403846,0.5,0.000000,0.181818,0.0,0.204301,0.727273,0.181818,0.454545,0.857143,0.727273,0.0,0.727273,0.619048,0.000000,0.000026,0.787234,0.392405,0.343434,0.000000,0.033333,0.777778,0.013840,0.000000,0.000000,0.000000,0.0,0.0,0.130435,0.040816,0.030303,0.422535,0.303030,0.000000,0.171717,0.200000,0.121212,0.216216,0.095238,0.08,0.000000,0.979798,0.380952,1.0,0.469388,0.294118,0.195122,0.006667,0.010050,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,0.596154,0.5,0.666667,0.363636,0.0,0.494624,0.000000,0.545455,0.545455,0.857143,0.909091,0.5,0.181818,0.714286,0.818182,0.000026,0.510638,0.367089,0.161616,0.083333,0.050000,1.000000,0.020427,0.000000,0.000000,0.000000,0.0,0.0,0.043478,0.102041,0.111111,0.056338,0.000000,0.000000,0.000000,0.418182,0.181818,0.135135,0.000000,0.08,0.014286,1.000000,0.285714,0.4,0.224490,0.294118,0.024390,0.011111,0.010050,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
965,0.500000,1.0,0.333333,0.636364,0.0,0.612903,0.000000,0.636364,0.000000,0.785714,0.727273,0.0,0.272727,0.857143,0.000000,0.025691,0.393617,0.531646,0.222222,0.145833,0.183333,0.777778,0.040522,0.000000,0.000000,0.018182,0.0,0.0,0.021739,0.061224,0.202020,0.056338,0.000000,0.000000,0.252525,0.672727,0.131313,0.081081,0.000000,0.20,0.028571,1.000000,0.333333,0.6,0.306122,0.294118,0.048780,0.011111,0.010050,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [43]:
lm2 = linear_model.LinearRegression()
lm2.fit(X_train_new_d,y_train_d)

predictions_d = lm2.predict(X_train_new_d)
print(r2_score(y_train_d, predictions_d))
predictions_test_d = lm2.predict(X_test_new_d)
print(r2_score(y_test_d, predictions_test_d))
mse = mean_squared_error(y_test_d,predictions_test_d)
rmse = np.sqrt(mean_squared_error(y_test_d,predictions_test_d))
print(rmse)
print(mse)

0.4104475907701173
0.44108945642393227
10.10017099253714
102.0134540784887
